In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
import torch.nn.functional as F
from tqdm import tqdm
import torch
import numpy as np
from sklearn.metrics import roc_auc_score,accuracy_score, roc_curve,auc,average_precision_score,precision_recall_curve
import cv2
from dependency import *
import pandas as pd
from utils import encode_test_label,Logger,encode_meta_choosed_label,encode_meta_label
from tqdm import tqdm_notebook  

test_index_df = pd.read_csv(test_index_path)
train_index_df = pd.read_csv(train_index_path)
val_index_df = pd.read_csv(val_index_path)

train_index_list = list(train_index_df['indexes'])
val_index_list = list(val_index_df['indexes'])
test_index_list = list(test_index_df['indexes'])

train_index_list_1 = train_index_list[0:206]
train_index_list_2 = train_index_list[206:]

df = pd.read_csv(img_info_path)

In [ ]:
from tensorflow.keras.utils import to_categorical
from second_stage_fusion_utils_concat import predict,predict3
import pandas as pd
from dependency import *
import torch
from model import Resnet_with_uncertainty

In [ ]:
def get_label_list(image_index_list):
    diag_label_list = []
    pn_label_list = []
    str_label_list = []
    pig_label_list = []
    rs_label_list = []
    dag_label_list = []
    bwv_label_list = []
    vs_label_list = []
    meta_list = []

    img_feature = []
    img_hf_feature = []
    img_vf_feature = []
    img_vhf_feature = []

    from sklearn.decomposition import PCA

    from tqdm import tqdm_notebook,tqdm
    for index_num in tqdm(image_index_list):
    #index_num = test_index_list[100]
        img_info = df[index_num:index_num+1]
        clinic_path = img_info['clinic']
        dermoscopy_path = img_info['derm']
        source_dir = '../release_v0/release_v0/images/'
        clinic_img = cv2.imread(source_dir+clinic_path[index_num])
        dermoscopy_img = cv2.imread(source_dir+dermoscopy_path[index_num])
        '''这里对于metadata需要变更'''
        meta_vector = encode_meta_choosed_label(img_info,index_num)

        [diagnosis_label,pigment_network_label,streaks_label,pigmentation_label,regression_structures_label,
         dots_and_globules_label,blue_whitish_veil_label, vascular_structures_label],[diagnosis_label_one_hot,pigment_network_label_one_hot,
        streaks_label_one_hot,pigmentation_label_one_hot,regression_structures_label_one_hot,
        dots_and_globules_label_one_hot,blue_whitish_veil_label_one_hot, vascular_structures_label_one_hot] = encode_test_label(img_info,index_num)

        diag_label_list.append(diagnosis_label)
        pn_label_list.append(pigment_network_label)
        str_label_list.append(streaks_label)
        pig_label_list.append(pigmentation_label)
        rs_label_list.append(regression_structures_label)
        dag_label_list.append(dots_and_globules_label)
        bwv_label_list.append(blue_whitish_veil_label)
        vs_label_list.append(vascular_structures_label)
        meta_list.append(meta_vector)



    label_dict ={'diag':diag_label_list,
                 'pn':pn_label_list,
                 'str':str_label_list,
                 'pig':pig_label_list,
                 'rs':rs_label_list,
                 'dag':dag_label_list,
                 'bwv':bwv_label_list,
                 'vs':vs_label_list}
    
    return label_dict,meta_list

In [ ]:
def prediction_2_weight_search(vs_prob2,val_vs_preds_prob,val_img_vs_label,tag='searched'):
    weight_list = []
    acc_list    = []
    if tag == 'mean':
        vs_prob_ = 0.5*vs_prob2+0.5*val_vs_preds_prob
        val_vs_preds_fusion = np.argmax(vs_prob_,1)
        vs_acc = np.mean(val_vs_preds_fusion==val_img_vs_label)
        weight_list.append(0.5)
        acc_list.append(vs_acc)
        best_weight = 0.5
        best_acc = vs_acc
    
    else:
        for i in np.linspace(0,1,num=101):
            vs_prob_ = i*vs_prob2+(1-i)*val_vs_preds_prob
            val_vs_preds_fusion = np.argmax(vs_prob_,1)
            vs_acc = np.mean(val_vs_preds_fusion==val_img_vs_label)
            #print("i:",i,vs_acc)
            weight_list.append(i)
            acc_list.append(vs_acc)
        
            index = np.argmax(acc_list)
            best_weight = weight_list[index]
            best_acc    = acc_list[index]
    return weight_list,acc_list,best_weight,best_acc

In [ ]:
def save_gt_result(plot_dir,gt_list,prob_list):
    
    gt,pn_gt,str_gt,pig_gt,rs_gt,dag_gt,bwv_gt,vs_gt = gt_list
    prob,pn_prob,str_prob,pig_prob,rs_prob,dag_prob,bwv_prob,vs_prob = prob_list    
    
    os.makedirs(plot_dir,exist_ok=True)
    
    np.save(plot_dir+'gt.npy',gt)
    np.save(plot_dir+'pn_gt.npy',pn_gt)
    np.save(plot_dir+'str_gt.npy',str_gt)
    np.save(plot_dir+'pig_gt.npy',pig_gt)
    np.save(plot_dir+'rs_gt.npy',rs_gt)
    np.save(plot_dir+'dag_gt.npy',dag_gt)
    np.save(plot_dir+'bwv_gt.npy',bwv_gt)
    np.save(plot_dir+'vs_gt.npy',vs_gt)
    
    print(vs_prob.shape)
    np.save(plot_dir+'prob.npy',prob)
    np.save(plot_dir+'pn_prob.npy',pn_prob)
    np.save(plot_dir+'str_prob.npy',str_prob)
    np.save(plot_dir+'pig_prob.npy',pig_prob)
    np.save(plot_dir+'rs_prob.npy',rs_prob)
    np.save(plot_dir+'dag_prob.npy',dag_prob)
    np.save(plot_dir+'bwv_prob.npy',bwv_prob)
    np.save(plot_dir+'vs_prob.npy',vs_prob)    

In [ ]:
#multi-classification

from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import numpy as np

def multi_classifier_predict(test_meta_feature):
    forest = RandomForestClassifier(random_state=1)
    clf = MultiOutputClassifier(forest,n_jobs=2)
    
    meta_label = np.array([img_diag_label,img_pn_label,img_str_label,img_pig_label,img_rs_label,img_dag_label,img_bwv_label,img_vs_label]).T
    
    clf.fit(meta_total_feature,meta_label)
    test_preds_all = clf.predict(test_meta_feature)
    print("test_preds_all.shape:",test_preds_all.shape)
    
    test_preds_prob_all = clf.predict_proba(test_meta_feature)
    rows = test_meta_feature.shape[0]
    print("rows:",rows)
   
    test_preds = test_preds_all[:,0]
    pn_test_preds = test_preds_all[:,1]
    str_test_preds = test_preds_all[:,2]
    pig_test_preds = test_preds_all[:,3]
    rs_test_preds = test_preds_all[:,4]
    dag_test_preds = test_preds_all[:,5]
    bwv_test_preds = test_preds_all[:,6]
    vs_test_preds = test_preds_all[:,7]
    
    test_preds_prob = np.array(test_preds_prob_all[0])
    pn_test_preds_prob = np.array(test_preds_prob_all[1])
    str_test_preds_prob = np.array(test_preds_prob_all[2])
    pig_test_preds_prob = np.array(test_preds_prob_all[3])
    rs_test_preds_prob = np.array(test_preds_prob_all[4])
    dag_test_preds_prob = np.array(test_preds_prob_all[5])
    bwv_test_preds_prob = np.array(test_preds_prob_all[6])
    vs_test_preds_prob = np.array(test_preds_prob_all[7])
    
    return [[test_preds,pn_test_preds,str_test_preds,pig_test_preds,rs_test_preds,dag_test_preds,bwv_test_preds,vs_test_preds],
            [test_preds_prob,pn_test_preds_prob,str_test_preds_prob,pig_test_preds_prob,rs_test_preds_prob,dag_test_preds_prob,bwv_test_preds_prob,vs_test_preds_prob]]

In [8]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
seed = 0
n_features = 20
search_model = ['Resnet_cos-loss and concat']
mode = 'multimodal'
model_name = 'Resnet_cos-loss and concat'
search_num = 100 #
import random
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
TTA = 6
img_type="concat"
tag = "searched"

size = 224
round_ = 1
data_mode = 'Normal'
#for i in range(round_):
#i = 55
p1_p2_fused_mode = 'searched'

p1_acc_list = []
p2_acc_list = []
p3_acc_list = []

mean_avg_acc_list = []

############################get test dataset predictions
for j in range(round_):
    i =  100+j
    weight_file = './{}_{}_{}_weight_file/{}/checkpoint/best_mean_acc_model.pth'.format(mode,model_name,data_mode,j)
    #weight_file = './weight/{}/checkpoint/best_mean_acc_model.pth'.format(j)

    print(weight_file)
    net = Resnet_with_uncertainty(class_list).cuda()
    out_dir = './{}_{}_result/{}/'.format(mode,model_name, i)
    net.load_state_dict(torch.load(weight_file))
    
    (avg_acc,
     [prob,pn_prob,str_prob,pig_prob,rs_prob,dag_prob,bwv_prob,vs_prob], 
     [nevu_label, bcc_label, mel_label, misc_label, sk_label],
     [nevu_prob ,bcc_prob ,  mel_prob ,misc_prob ,sk_prob],
     seven_point_feature_list,
     [gt,pn_gt,str_gt,pig_gt,rs_gt,dag_gt, bwv_gt,vs_gt] )= predict(net,train_index_list,df,
                                                                    model_name,out_dir,mode,TTA,size,img_type,"img")
    seven_point_feature_list_train = seven_point_feature_list
    
    print('predicting on testing dataset....P_1')
    (avg_acc,
     [prob,pn_prob,str_prob,pig_prob,rs_prob,dag_prob,bwv_prob,vs_prob], 
     [nevu_label, bcc_label, mel_label, misc_label, sk_label],
     [nevu_prob ,bcc_prob ,  mel_prob ,misc_prob ,sk_prob],
     seven_point_feature_list,
     [gt,pn_gt,str_gt,pig_gt,rs_gt,dag_gt, bwv_gt,vs_gt])= predict(net,test_index_list,df,
                                                                    model_name,out_dir,mode,TTA,size,img_type,"img")

    p1_acc_list.append(avg_acc)
    prob_1 = prob
    pn_prob_1 = pn_prob
    str_prob_1 = str_prob
    pig_prob_1 = pig_prob
    rs_prob_1  = rs_prob
    dag_prob_1 = dag_prob
    bwv_prob_1 = bwv_prob
    vs_prob_1  = vs_prob
    seven_point_feature_list_p1 = seven_point_feature_list

    test_label_list = [gt,pn_gt,str_gt,pig_gt,rs_gt,dag_gt, bwv_gt,vs_gt]
    p1_list  = [prob,pn_prob,str_prob,pig_prob,rs_prob,dag_prob,bwv_prob,vs_prob]

    print('Done!')

    #################get validation dataset predictions#####################
    print('predicting on validation dataset....P_val')


    (avg_acc,
     [prob,pn_prob,str_prob,pig_prob,rs_prob,dag_prob,bwv_prob,vs_prob], 
     [nevu_label, bcc_label, mel_label, misc_label, sk_label],
     [nevu_prob ,bcc_prob ,  mel_prob ,misc_prob ,sk_prob],
     seven_point_feature_list,
     [gt,pn_gt,str_gt,pig_gt,rs_gt,dag_gt, bwv_gt,vs_gt] )= predict(net,val_index_list,df,
                                                                    model_name,out_dir,mode,TTA,size,img_type,"img")

    prob_val = prob
    pn_prob_val = pn_prob
    str_prob_val = str_prob
    pig_prob_val = pig_prob
    rs_prob_val  = rs_prob
    dag_prob_val = dag_prob
    bwv_prob_val = bwv_prob
    vs_prob_val  = vs_prob

    seven_point_feature_list_val = seven_point_feature_list
    print('Done!')
    
 #################fusing predictions from two-modality images and metadata by concatenation#####################

    #################get labels#####################
    print('fusing predictions from two-modality images and metadata......')
    train_label_dict,train_meta_list = get_label_list(train_index_list)
    val_label_dict,val_meta_list   = get_label_list(val_index_list)
    test_label_dict,test_meta_list  = get_label_list(test_index_list)


############################这里的P2应该改成metadata直接通过一个multi-classification输出的P2#####################
#############################################然后后续是P2和P1的search##############################################

    train_meta_array = np.array(train_meta_list)
    print("train_meta_array.shape:",train_meta_array.shape)
    print("seven_point_feature_list_train.shape:",np.array(seven_point_feature_list_train).shape)
    
    meta_total_feature = train_meta_array
    print(meta_total_feature.shape)
    
    train_diag_label = np.array(train_label_dict['diag'])
    print(train_diag_label.shape)

    val_meta_array = np.array(val_meta_list)
    val_meta_total_feature = val_meta_array
    print(val_meta_total_feature.shape)
    
    val_diag_label = np.array(val_label_dict['diag'])
    print(val_diag_label.shape)

    test_meta_array = np.array(test_meta_list)
    test_meta_total_feature = test_meta_array
    print(test_meta_total_feature.shape)
    
    test_diag_label = np.array(test_label_dict['diag'])
    print(test_diag_label.shape)
    print('Done!')

    img_pn_label= np.array(train_label_dict['pn'])
    img_str_label= np.array(train_label_dict['str'])
    img_pig_label= np.array(train_label_dict['pig'])
    img_rs_label= np.array(train_label_dict['rs'])
    img_dag_label= np.array(train_label_dict['dag'])
    img_bwv_label= np.array(train_label_dict['bwv'])
    img_vs_label= np.array(train_label_dict['vs'])
    img_diag_label = np.array(train_label_dict['diag'])

    test_img_pn_label= np.array(test_label_dict['pn'])
    test_img_str_label= np.array(test_label_dict['str'])
    test_img_pig_label= np.array(test_label_dict['pig'])
    test_img_rs_label= np.array(test_label_dict['rs'])
    test_img_dag_label= np.array(test_label_dict['dag'])
    test_img_bwv_label= np.array(test_label_dict['bwv'])
    test_img_vs_label= np.array(test_label_dict['vs'])
    test_img_diag_label = np.array(test_label_dict['diag'])


    val_img_pn_label= np.array(val_label_dict['pn'])
    val_img_str_label= np.array(val_label_dict['str'])
    val_img_pig_label= np.array(val_label_dict['pig'])
    val_img_rs_label= np.array(val_label_dict['rs'])
    val_img_dag_label= np.array(val_label_dict['dag'])
    val_img_bwv_label= np.array(val_label_dict['bwv'])
    val_img_vs_label= np.array(val_label_dict['vs'])
    val_img_diag_label= np.array(val_label_dict['diag'])


    img_pn_label_one_hot = to_categorical(np.array(train_label_dict['pn']))
    img_str_label_one_hot= to_categorical(np.array(train_label_dict['str']))
    img_pig_label_one_hot= to_categorical(np.array(train_label_dict['pig']))
    img_rs_label_one_hot= to_categorical(np.array(train_label_dict['rs']))
    img_dag_label_one_hot= to_categorical(np.array(train_label_dict['dag']))
    img_bwv_label_one_hot= to_categorical(np.array(train_label_dict['bwv']))
    img_vs_label_one_hot= to_categorical(np.array(train_label_dict['vs']))
    img_diag_label_one_hot = to_categorical(np.array(train_label_dict['diag']))

    test_img_pn_label_one_hot = to_categorical(np.array(test_label_dict['pn']))
    test_img_str_label_one_hot= to_categorical(np.array(test_label_dict['str']))
    test_img_pig_label_one_hot= to_categorical(np.array(test_label_dict['pig']))
    test_img_rs_label_one_hot= to_categorical(np.array(test_label_dict['rs']))
    test_img_dag_label_one_hot= to_categorical(np.array(test_label_dict['dag']))
    test_img_bwv_label_one_hot= to_categorical(np.array(test_label_dict['bwv']))
    test_img_vs_label_one_hot= to_categorical(np.array(test_label_dict['vs']))
    test_img_diag_label_one_hot = to_categorical(np.array(test_label_dict['diag']))

    print('Training multi-Classifier.....')
    _,[prob_2,pn_prob_2,str_prob_2,pig_prob_2,rs_prob_2,dag_prob_2,bwv_prob_2,vs_prob_2] = multi_classifier_predict(test_meta_total_feature)
    _,[val_p2_prob,val_p2_pn_prob,val_p2_str_prob,val_p2_pig_prob,val_p2_rs_prob,val_p2_dag_prob,val_p2_bwv_prob,val_p2_vs_prob] = multi_classifier_predict(val_meta_total_feature)
    print('training Done!')
    p2_list  = [prob_2,pn_prob_2,str_prob_2,pig_prob_2,rs_prob_2,dag_prob_2,bwv_prob_2,vs_prob_2]
    plot_dir_p2 = './{}_{}_p2_plot_figure/{}/'.format( mode,model_name,i)
    save_gt_result(plot_dir_p2,test_label_list,p2_list)

    pn_test_preds = np.argmax(pn_prob_2,1)
    str_test_preds = np.argmax(str_prob_2,1)
    pig_test_preds = np.argmax(pig_prob_2,1)
    rs_test_preds = np.argmax(rs_prob_2,1)
    dag_test_preds = np.argmax(dag_prob_2,1)
    bwv_test_preds = np.argmax(bwv_prob_2,1)
    vs_test_preds = np.argmax(vs_prob_2,1)
    diag_test_preds = np.argmax(prob_2,1)
    
    
    pn_acc = np.mean(pn_test_preds==test_img_pn_label)
    str_acc = np.mean(str_test_preds==test_img_str_label)
    pig_acc = np.mean(pig_test_preds==test_img_pig_label)
    rs_acc = np.mean(rs_test_preds==test_img_rs_label)
    dag_acc = np.mean(dag_test_preds==test_img_dag_label)
    bwv_acc = np.mean(bwv_test_preds==test_img_bwv_label)
    vs_acc = np.mean(vs_test_preds==test_img_vs_label)
    diag_acc = np.mean(diag_test_preds==test_img_diag_label)
    avg_acc = (pn_acc + str_acc + pig_acc + rs_acc + dag_acc + bwv_acc + vs_acc + diag_acc) / 8

    
    print('multi_classification on test-data of P2--------------------------')
    print('avg_acc : {}'.format(avg_acc))
    print('vs_acc : {}'.format(vs_acc))
    print('bwv_acc : {}'.format(bwv_acc))
    print('dag_acc : {}'.format(dag_acc))
    print('rs_acc : {}'.format(rs_acc))
    print('pig_acc : {}'.format(pig_acc))
    print('str_acc : {}'.format(str_acc))
    print('pn_acc : {}'.format(pn_acc))
    print('diag_acc : {}'.format(diag_acc))
    
    (avg_acc,
     [prob,pn_prob,str_prob,pig_prob,rs_prob,dag_prob,bwv_prob,vs_prob], 
     [nevu_label, bcc_label, mel_label, misc_label, sk_label],
     [nevu_prob ,bcc_prob ,  mel_prob ,misc_prob ,sk_prob],
     seven_point_feature_list,
     [gt,pn_gt,str_gt,pig_gt,rs_gt,dag_gt, bwv_gt,vs_gt] )= predict(net,test_index_list,df,
                                                                    model_name,out_dir,mode,TTA,size,img_type,"metadata")
    prob_test = prob
    print("prob_test.shape:",prob_test.shape)
    pn_prob_test = pn_prob
    str_prob_test = str_prob
    pig_prob_test = pig_prob
    rs_prob_test  = rs_prob
    dag_prob_test = dag_prob
    bwv_prob_test = bwv_prob
    vs_prob_test  = vs_prob

    seven_point_feature_list_test = seven_point_feature_list
    print('Done!')
    print("P2 done!")
    
    vs_weight_list,vs_acc_list,vs_best_weight,vs_best_acc = prediction_2_weight_search(vs_prob_val,val_p2_vs_prob,val_img_vs_label,tag)
    str_weight_list,str_acc_list,str_best_weight,str_best_acc = prediction_2_weight_search(str_prob_val,val_p2_str_prob,val_img_str_label,tag)
    pn_weight_list,pn_acc_list,pn_best_weight,pn_best_acc = prediction_2_weight_search(pn_prob_val,val_p2_pn_prob,val_img_pn_label,tag)
    pig_weight_list,pig_acc_list,pig_best_weight,pig_best_acc = prediction_2_weight_search(pig_prob_val,val_p2_pig_prob,val_img_pig_label,tag)
    bwv_weight_list,bwv_acc_list,bwv_best_weight,bwv_best_acc = prediction_2_weight_search(bwv_prob_val,val_p2_bwv_prob,val_img_bwv_label,tag)
    dag_weight_list,dag_acc_list,dag_best_weight,dag_best_acc = prediction_2_weight_search(dag_prob_val,val_p2_dag_prob,val_img_dag_label,tag)
    rs_weight_list,rs_acc_list,rs_best_weight,rs_best_acc = prediction_2_weight_search(rs_prob_val,val_p2_rs_prob,val_img_rs_label,tag)
    pred_weight_list,pred_acc_list,pred_best_weight,pred_best_acc = prediction_2_weight_search(prob_val,val_p2_prob,val_img_diag_label,tag)

    vs_arr = np.array(vs_acc_list)
    str_arr = np.array(str_acc_list)
    pn_arr = np.array(pn_acc_list)
    pig_arr = np.array(pig_acc_list)
    bwv_arr = np.array(bwv_acc_list)
    dag_arr = np.array(dag_acc_list)
    rs_arr = np.array(rs_acc_list)
    pred_arr = np.array(pred_acc_list)

    avg_arr = (vs_arr+str_arr+pn_arr+pig_arr+bwv_arr+dag_arr+rs_arr+pred_arr)/8
    index = np.argmax(avg_arr)

    print('searched P1-P2 fused weights----')
    print(pred_weight_list[index])
    print("pred_best_weight:",pred_best_weight)
    print(avg_arr[index])

    best_weight = pred_weight_list[index]
    print("best_weight:",best_weight)
    p1_p2_weight_list = [0.5,best_weight]
    p1_p2_fused_mode_list = ['average','searched']
    
    for index in range(len(p1_p2_weight_list)):

        best_weight = p1_p2_weight_list[index]
        p1_p2_fused_mode = p1_p2_fused_mode_list[index]

        prob_3 = (best_weight*prob_1+(1-best_weight)*prob_2)
        pred_3 = np.argmax(prob_3,1)

        pn_prob_3 = (best_weight*pn_prob_1 + (1-best_weight)*pn_prob_2)
        pn_pred_3 = np.argmax(pn_prob_3,1)

        pig_prob_3 = (best_weight*pig_prob_1 +(1-best_weight)*pig_prob_2)
        pig_pred_3 = np.argmax(pig_prob_3,1)

        str_prob_3 = (best_weight*str_prob_1 + (1-best_weight)*str_prob_2)
        str_pred_3 = np.argmax(str_prob_3,1)

        rs_prob_3 = (best_weight*rs_prob_1 + (1-best_weight)*rs_prob_2)
        rs_pred_3 = np.argmax(rs_prob_3,1)

        bwv_prob_3 = (best_weight*bwv_prob_1 + (1-best_weight)*bwv_prob_2)
        bwv_pred_3 = np.argmax(bwv_prob_3,1)

        vs_prob_3 = (best_weight*vs_prob_1 + (1-best_weight)*vs_prob_2)
        vs_pred_3 = np.argmax(vs_prob_3,1)

        dag_prob_3 = (best_weight*dag_prob_1 + (1-best_weight)*dag_prob_2)
        dag_pred_3 = np.argmax(dag_prob_3,1)

        pn_acc = np.mean(pn_pred_3==test_img_pn_label)
        str_acc = np.mean(str_pred_3==test_img_str_label)
        pig_acc = np.mean(pig_pred_3==test_img_pig_label)
        rs_acc = np.mean(rs_pred_3==test_img_rs_label)
        dag_acc = np.mean(dag_pred_3==test_img_dag_label)
        bwv_acc = np.mean(bwv_pred_3==test_img_bwv_label)
        vs_acc = np.mean(vs_pred_3==test_img_vs_label)
        diag_acc = np.mean(pred_3==test_img_diag_label)

        avg_acc = (pn_acc + str_acc + pig_acc + rs_acc + dag_acc + bwv_acc + vs_acc + diag_acc) / 8
        mean_avg_acc_list.append(avg_acc)
        p3_list  = [prob_3,pn_prob_3,str_prob_3,pig_prob_3,rs_prob_3,dag_prob_3,bwv_prob_3,vs_prob_3]
        plot_dir_p3 = './{}_{}_{}_{}_p3_plot_figure/{}/'.format(model_name, mode,p1_p2_fused_mode, "Linear",i)
        save_gt_result(plot_dir_p3,test_label_list,p3_list)
        print('p1_p2_fused_mode : {}'.format(p1_p2_fused_mode))
        print('P3--------------------------')
        print('avg_acc : {}'.format(avg_acc))
        print('vs_acc : {}'.format(vs_acc))
        print('bwv_acc : {}'.format(bwv_acc))
        print('dag_acc : {}'.format(dag_acc))
        print('rs_acc : {}'.format(rs_acc))
        print('pig_acc : {}'.format(pig_acc))
        print('str_acc : {}'.format(str_acc))
        print('pn_acc : {}'.format(pn_acc))
        print('diag_acc : {}'.format(diag_acc))
        (avg_acc,
     [prob,pn_prob,str_prob,pig_prob,rs_prob,dag_prob,bwv_prob,vs_prob], 
     [nevu_label, bcc_label, mel_label, misc_label, sk_label],
     [nevu_prob ,bcc_prob ,  mel_prob ,misc_prob ,sk_prob],
     seven_point_feature_list,
     [gt,pn_gt,str_gt,pig_gt,rs_gt,dag_gt, bwv_gt,vs_gt] )= predict3(net,test_index_list,df,
                                                                    model_name,out_dir,mode,best_weight,TTA,size,img_type)
print(mean_avg_acc_list)
#averaged_avg_acc = np.mean([mean_avg_acc_list[0],mean_avg_acc_list[2],mean_avg_acc_list[4],mean_avg_acc_list[6],mean_avg_acc_list[8]])
#searched_avg_acc = np.mean([mean_avg_acc_list[1],mean_avg_acc_list[3],mean_avg_acc_list[5],mean_avg_acc_list[7],mean_avg_acc_list[9]])

#print('averaged avg acc: {}'.format(averaged_avg_acc))
#print('searched avg acc: {}'.format(searched_avg_acc))


./multimodal_Resnet_cos-loss and concat_Normal_weight_file/0/checkpoint/best_mean_acc_model.pth

--- [START IDENTIFIER] ----------------------------------------------------------------




100%|████████████████████████████████████████████████████████████████████████████████| 413/413 [00:46<00:00,  8.86it/s]


---------------

avg_acc : 0.8837772397094431

vs_acc : 0.9079903147699758

bwv_acc : 0.9709443099273608

dag_acc : 0.7772397094430993

rs_acc : 0.9443099273607748

pig_acc : 0.8861985472154964

str_acc : 0.9152542372881356

pn_acc : 0.8087167070217918

diag_acc : 0.8595641646489104

---------------

nevu_auc: 0.9764629777070063

bcc_auc: 0.9703446433342238

mel_auc: 0.9855865153078776

misc_auc: 0.9665354330708662

sk_auc: 0.9707178841309825

---------------

vs_asb_auc: 0.9827089337175793

vs_reg_auc: 0.976869893148963

vs_irg_auc: 0.9566332218506132

---------------

bwv_asb_auc: 0.9916287969385315

bwv_prs_auc: 0.9916287969385315

---------------

dag_asb_auc: 0.9341438703140831

dag_reg_auc: 0.9181382819515116

dag_irg_auc: 0.9348747591522157

---------------

rs_asb_auc: 0.9876445846477392

rs_prs_auc: 0.9876445846477393

---------------

pig_asb_auc: 0.9759881422924902

pig_reg_auc: 0.9815841340231584

pig_irg_auc: 0.9686230117264599

---------------

str_asb_auc: 0.992595499738

100%|████████████████████████████████████████████████████████████████████████████████| 395/395 [00:43<00:00,  9.11it/s]


---------------

avg_acc : 0.7259493670886076

vs_acc : 0.8177215189873418

bwv_acc : 0.8860759493670886

dag_acc : 0.6

rs_acc : 0.7848101265822784

pig_acc : 0.6784810126582278

str_acc : 0.7265822784810126

pn_acc : 0.6329113924050633

diag_acc : 0.6810126582278481

---------------

nevu_auc: 0.8781392694063925

bcc_auc: 0.8848944591029023

mel_auc: 0.880514582070452

misc_auc: 0.8790845070422535

sk_auc: 0.7371220604703248

---------------

vs_asb_auc: 0.8685810021039508

vs_reg_auc: 0.8605068400986768

vs_irg_auc: 0.8092237442922374

---------------

bwv_asb_auc: 0.9197083333333333

bwv_prs_auc: 0.9197083333333333

---------------

dag_asb_auc: 0.762406779661017

dag_reg_auc: 0.8080523771645353

dag_irg_auc: 0.7877727673249364

---------------

rs_asb_auc: 0.822843898935823

rs_prs_auc: 0.8228438989358229

---------------

pig_asb_auc: 0.8072270309729899

pig_reg_auc: 0.7771974063400576

pig_irg_auc: 0.8193072253303179

---------------

str_asb_auc: 0.8490103197428523

str_reg_auc

...

train_meta_array.shape: (413, 18)
seven_point_feature_list_train.shape: (413, 24)
(413, 18)
(413,)
(203, 18)
(203,)
(395, 18)
(395,)
Done!
Training multi-Classifier.....
test_preds_all.shape: (395, 8)
rows: 395
test_preds_all.shape: (203, 8)
rows: 203
training Done!
(395, 3)
multi_classification on test-data of P2--------------------------
avg_acc : 0.6889240506329113
vs_acc : 0.759493670886076
bwv_acc : 0.8455696202531645
dag_acc : 0.5873417721518988
rs_acc : 0.7063291139240506
pig_acc : 0.5594936708860759
str_acc : 0.6987341772151898
pn_acc : 0.6025316455696202
diag_acc : 0.7518987341772152

--- [START IDENTIFIER] ----------------------------------------------------------------




100%|████████████████████████████████████████████████████████████████████████████████| 395/395 [03:26<00:00,  1.91it/s]


---------------

avg_acc : 0.6889240506329113

vs_acc : 0.759493670886076

bwv_acc : 0.8455696202531645

dag_acc : 0.5873417721518988

rs_acc : 0.7063291139240506

pig_acc : 0.5594936708860759

str_acc : 0.6987341772151898

pn_acc : 0.6025316455696202

diag_acc : 0.7518987341772152

---------------

nevu_auc: 0.8700316521378165

bcc_auc: 0.8238786279683377

mel_auc: 0.8856671381423857

misc_auc: 0.8353873239436619

sk_auc: 0.8359462486002239

---------------

vs_asb_auc: 0.6531987843840099

vs_reg_auc: 0.6660125588697017

vs_irg_auc: 0.6893150684931507

---------------

bwv_asb_auc: 0.8370416666666667

bwv_prs_auc: 0.8370416666666668

---------------

dag_asb_auc: 0.6168644067796609

dag_reg_auc: 0.7389249219849477

dag_irg_auc: 0.7932151557559737

---------------

rs_asb_auc: 0.7012959456812691

rs_prs_auc: 0.7012959456812692

---------------

pig_asb_auc: 0.5726744186046511

pig_reg_auc: 0.6413604707012488

pig_irg_auc: 0.6719140578502559

---------------

str_asb_auc: 0.786231884057

  0%|                                                                                          | 0/395 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 8)